In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

In [2]:

from azure.identity import DefaultAzureCredential, get_bearer_token_provider

In [3]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import BingGroundingTool

In [4]:
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

In [5]:
az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment="gpt-4o-mini",
    api_version="2024-05-01-preview",
    model = "gpt-4o-mini",
    azure_endpoint="Your AOAI endpoint",
    azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    # api_key="sk-...", # For key-based authentication.
)

In [6]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str='Your Azure AI Foundation Connection String',
)

In [7]:
bing_connection = project_client.connections.get(
    connection_name='kinfey-bing-search'
)

conn_id = bing_connection.id

In [8]:
async def web_ai_agent(query: str) -> str:
    print("This is Bing for Azure AI Agent Service .......")
    bing = BingGroundingTool(connection_id=conn_id)
    with project_client:
        agent = project_client.agents.create_agent(
            model="gpt-4",
            name="my-assistant",
            instructions="""        
                You are a web search agent.
                Your only tool is search_tool - use it to find information.
                You make only one search call at a time.
                Once you have the results, you never do calculations based on them.
            """,
            tools=bing.definitions,
            headers={"x-ms-enable-preview": "true"}
        )
        print(f"Created agent, ID: {agent.id}")

        # Create thread for communication
        thread = project_client.agents.create_thread()
        print(f"Created thread, ID: {thread.id}")

        # Create message to thread
        message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=query,
        )
        print(f"SMS: {message}")
        # Create and process agent run in thread with tools
        run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
        print(f"Run finished with status: {run.status}")

        if run.status == "failed":
            print(f"Run failed: {run.last_error}")

        # Delete the assistant when done
        project_client.agents.delete_agent(agent.id)
        print("Deleted agent")

        # Fetch and log all messages
        messages = project_client.agents.list_messages(thread_id=thread.id)
        print("Messages:"+ messages["data"][0]["content"][0]["text"]["value"])
    return messages["data"][0]["content"][0]["text"]["value"]



In [9]:
bing_search_agent = AssistantAgent(
    name="assistant",
    model_client=az_model_client,
    tools=[web_ai_agent],
    system_message="Use tools to solve tasks.",
)

In [10]:
bing = BingGroundingTool(connection_id=conn_id)

In [11]:
async def assistant_run() -> None:
    response = await bing_search_agent.on_messages(
            [TextMessage(content="GitHub Copilot 是什么", source="user")],
            cancellation_token=CancellationToken(),
    )
#     print(response.inner_messages)
    print(response.chat_message)

In [12]:
await assistant_run()

This is Bing for Azure AI Agent Service .......
Created agent, ID: asst_mpVMN2ydkGXtWstvPXIrYZiE
Created thread, ID: thread_R4QGk0kKoQIhrHu3hHF5A2K8
SMS: {'id': 'msg_DSk03jjbffz8DBqrcHZY7uDh', 'object': 'thread.message', 'created_at': 1735034344, 'assistant_id': None, 'thread_id': 'thread_R4QGk0kKoQIhrHu3hHF5A2K8', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'GitHub Copilot 是什么', 'annotations': []}}], 'attachments': [], 'metadata': {}}
Run finished with status: RunStatus.COMPLETED
Deleted agent
Messages:GitHub Copilot 是一个基于人工智能的编程辅助工具，由 GitHub 和 OpenAI 协同开发。它提供代码补全功能，可以帮助开发者更快地写出代码，并对各种编程语言的代码生产提供实时的建议和解决方案。Copilot 模型采用了大量公开可用的代码，包括开源项目中的代码，以训练其算法，能够理解和生成相对复杂的代码模式。这种工具特别适用于加速开发过程，减轻编程任务中的某些重复性劳动。
source='assistant' models_usage=None content='GitHub Copilot 是一个基于人工智能的编程辅助工具，由 GitHub 和 OpenAI 协同开发。它提供代码补全功能，可以帮助开发者更快地写出代码，并对各种编程语言的代码生产提供实时的建议和解决方案。Copilot 模型采用了大量公开可用的代码，包括开源项目中的代码，以训练其算法，能够理解和生成相对复杂的代码模式。这种工具特别适用于加速开发过程，减轻编程任务中的某些重复性劳动。' type='TextMessa